In [11]:
"""
 * @ Author: Lucas Glasner (lgvivanco96@gmail.com)
 * @ Create Time: 1969-12-31 21:00:00
 * @ Modified by: Lucas Glasner, 
 * @ Modified time: 2024-04-01 17:17:25
 * @ Description:
        Python translation of the "rational method" for computing peak runoff
        on drainage basins. 
 * @ Dependencies:
 */

"""
import pandas as pd
import numpy as np
import tqdm
import os

from src.rain import duration_coef
from src.infiltration import CN_correction
from src.geomorphology import tc_california, tc_giandotti, tc_kirpich
from src.geomorphology import tc_SCS, tc_spain
from src.misc import to_numeric

In [12]:
# ------------------------------- main routines ------------------------------ #
def parse_params(params_path):
    """
    Load input data and preprocess for the model run

    Args:
        params_path (str): Path to textfile or spreadsheet
                               with basin parameters. 

    Returns:
        (tuple): formatted parameters,
                 daily rainfall
    """    
    file_ext = params_path.split('.')[-1]
    if (file_ext == 'csv') or (file_ext == 'txt'):
        params = pd.read_csv(params_path, index_col=0)
    elif (file_ext == 'xlsx') or (file_ext == 'xls'):
        params = pd.read_excel(params_path, index_col=0)
    else:
        raise ValueError('Parameter extension file must be ".xlsx" or ".csv"')
    
    params = params.map(lambda x: to_numeric(x))
    
    pr_mm24hr = params[params.index.map(lambda x: 'pr' in x)]
    return_periods = pr_mm24hr.index.map(
        lambda x: int(x.split('_')[1].replace('T', '')))
    pr_mm24hr.index = return_periods
    
    params.loc['final_curvenumber_1'] = [CN_correction(cn, amc)
                                        for cn, amc in zip(
                                            params.loc['curvenumber_1'],
                                            params.loc['moisturecondition_1'])]
    params = params.loc[['name','area_km2', 'mriverlen_km', 'meanslope_1',
                         'hmax_m', 'hmin_m', 'hmed_m', 'runoffcoef_1',
                         'final_curvenumber_1']]
    return params, pr_mm24hr

def compute_tc(params):
    """
    Given the dataframe with basin parameters this function computes
    the concentration time with all the methods and merges them in a single
    table. If you want more methods for the concentration time just create 
    a new function like the above and add them in the dataframe that is built
    in here.

    Args:
        params (DataFrame): pandas DataFrame with basin parameters

    Returns:
        (DataFrame): Basin concentration times computed with different methods.
    """
    params = params.copy()
    # SCS concentration time
    basin_tc_SCS = tc_SCS(params.loc['mriverlen_km'],
                        params.loc['meanslope_1'],
                        params.loc['final_curvenumber_1'])

    # Kirpich concentration time
    basin_tc_kirpich = tc_kirpich(params.loc['mriverlen_km'],
                            params.loc['hmax_m']-params.loc['hmin_m'])

    # Giandotti concentration time
    basin_tc_giandotti = tc_giandotti(params.loc['mriverlen_km'],
                                    params.loc['hmed_m'],
                                    params.loc['area_km2'])

    # California concentration time
    basin_tc_california = tc_california(params.loc['mriverlen_km'],
                            params.loc['hmax_m']-params.loc['hmin_m'])

    # Spanish norms concentration time
    basin_tc_spain = tc_spain(params.loc['mriverlen_km'],
                              params.loc['meanslope_1'])
    
    basin_tcs = pd.concat([basin_tc_SCS,
                           basin_tc_kirpich,
                           basin_tc_giandotti,
                           basin_tc_california,
                           basin_tc_spain], axis=1)
    basin_tcs.columns = ['tc_SCS_hr',
                         'tc_kirpich_hr',
                         'tc_giandotti_hr',
                         'tc_california_hr',
                         'tc_spain_hr']
    return basin_tcs


def RationalMethod(parameters,
                   pr_mm24hr):
    """
    Given dataframes with basin parameters and precipitations this function
    computes the peak runoff of the rational method. 

    Args:
        parameters (DataFrame): pandas DataFrame with basin parameters
        pr_mm24hr (DataFrame): pandas DataFrame with precipitation for
            each basin and return period.

    Returns:
        basin_tcs, pr_mmXhr, peakrunoff (tuple):
            DataFrames with concentration time, precipitation intensity at the
            concentration time and model's peak runoff. 
    """
    # Runoff coefficient "frequency factors"
    runoff_FF  = pd.Series([1, 1, 1, 1.1, 1.2, 1.25, 1.275, 1.3],
                        index=[2,5,10,25,50,100,150,200])
    pr_mm24hr = pr_mm24hr.copy()
    parameters     = parameters.copy()
    # Compute concentration time and change units to hours
    basin_tcs = compute_tc(parameters)/60

    # Compute the duration coefficient for the concentration time
    DCoeffs = [duration_coef(basin_tcs.iloc[:,i].values)
                            for i in range(len(basin_tcs.columns))]
    DCoeffs = pd.DataFrame(DCoeffs,
                           columns=basin_tcs.index,
                           index=basin_tcs.columns)
    # Compute precipitation associated with the concentration time duration
    pr_mmXhr = pd.concat([pr_mm24hr*DCoeffs.loc[method]/basin_tcs[method]
                            for method in DCoeffs.index],
                            keys=DCoeffs.index)
    # Compute runoff coefficient for all basins and return periods
    runoff_coefs = [runoff_FF*parameters.loc['runoffcoef_1'].loc[basin]
                    for basin in parameters.columns]
    runoff_coefs = pd.concat(runoff_coefs, keys=parameters.columns).unstack().T

    # Compute peak runoff
    basin_areas  = parameters.loc['area_km2']
    peakrunoff   = [runoff_coefs*pr_mmXhr.loc[method]*basin_areas/3.6
                    for method in DCoeffs.index]
    peakrunoff   = pd.concat(peakrunoff, keys=DCoeffs.index)
    return basin_tcs.T, pr_mmXhr, peakrunoff

In [13]:
%%time
# ------------------------------ load input data ----------------------------- #
params, precips_mm24hr = parse_params('data/params.csv')

# ------------------------- run model for all basins ------------------------- #
basin_tcs, precips_mmXhr, peakrunoff = RationalMethod(params, precips_mm24hr)

# ------------------------ save results for each basin ----------------------- #
for fid in tqdm.tqdm(params.columns, total=len(params.columns)):
    try:
        os.mkdir(f'data/Basins/{name}')
        os.mkdir(f'data/Basins/{name}/tmp')
    except:
        pass
    name = params[fid].loc['name'].replace(' ','')
    out = f'data/Basins/{name}/QRational_{name}.xlsx'
    with pd.ExcelWriter(out, mode='w', engine='openpyxl') as writer:
        pd.concat([params[fid], basin_tcs[fid]]).to_excel(
            writer, sheet_name='concentration_time_hr')
        
    with pd.ExcelWriter(out, mode='a', engine='openpyxl') as writer:
        precips_mmXhr[fid].unstack().to_excel(
            writer, sheet_name='rainfall_intensity_mmhr-1')
        peakrunoff[fid].unstack().to_excel(
            writer, sheet_name='peakrunoff_m3s')

100%|██████████| 7/7 [00:01<00:00,  6.08it/s]

CPU times: user 514 ms, sys: 59.9 ms, total: 574 ms
Wall time: 1.22 s
